<br />    
<br />
<br />
# Results of chunking and compression mechanisms
<br />    
            
            
Using compression in HDF5 requires chunking. Chunking is the process of storing different subsets of the dataset contiguously on disk. For example for an array of dimensions (36, 257, 167, 84), chunking with (18, 257, 167, 84) gives four chunks. Chunking can be utilized when anticipating reading subsets of data. Compared to storing each row of an array conitguously on disk, reading subset of the array that match the chunking mechanism at the time of storage increases efficiency.

There are multiple variable that can control how the compression is done:

* chunks: The shape of the chunk
* compression: The compression algorithm, it can be either of gzip, szip, lzf, or blosc compressors such as zstd, blosclz, lz4, lz4hc, zlib
* compression_opts: The options for each compression algorithm
* shuffle: rearranging bytes in the data for possibility of improved compression

Compression and write speed:

The results in this section are dependent on the hardware. We start with considering only the Y channel. The analysis is similar for both channels.

Let's consider the following chunk shape, with 'gzip', at compresion option level 3, and with shuffle turned off:

|Chunk Size | Compression | Compression Options | Shuffle | Channel | File Size (MB) | Write Time (sec) | Read Time (sec) |
| -- | -- | -- | -- | -- | -- | -- | -- |
| (18, 36, 10, 21) |	gzip |	3|	0|	Y|	419.16|	45.48|	6.72|


After compressing the Y channel only, the file size is 419.16 MB Bytes and it takes 45.48 seconds for the data to be written, and 6.72 seconds to read the data back. We try turning the shuffling on:

|Chunk Size | Compression | Compression Options | Shuffle | Channel | File Size (MB) | Write Time (sec) | Read Time (sec) |
| -- | -- | -- | -- | -- | -- | -- | -- |
|(18, 36, 10, 21)|	gzip|	3|	1|	Y|	380.97 | 34.66 |	4.93 |

The shuffling algorithms takes advantage of the fact that numerical data in nearby voxels have reasonably close values. It reorders the bytes representing the values of nearby voxels, and places the zeros together. This allows for better compression.

Further testing shows that shuffling consistently gives significantly better results for other chunking sizes and algorithms. We will keep the shuffling on.

Testing with different settings with szip shows consistent lower compression ratio and longer write time. Among blosc algorithms, zstd and lz4 performed best on the available data. gzip and zstd give the best compression size. However, gzip has significantly slower read and write times compared to blosc algorithms and lzf. 


The following shows the results for different chunking sizes.

<img src="./media/Graph.png" width="90%" title="A"/>

The following shows the results for Iterative mode.

<img src="./media/Graph_Iterative.png" width="90%"/>


In [ ]:
# Setting up data for Benchmarking:

import os
from IPython.display import display
import scipy.io
import numpy as np
import hdf5plugin
import h5py


path_to_files = 'Folder/AxelLab/data'
# info file name in path_to_files folder
fname0 = 'fly2_run1_info.mat'
# .mat file containing Calcium Imaging data in path_to_files folder
fname1 = '2019_04_18_Nsyb_NLS6s_Su_walk_G_fly2_run1_8401reg.mat'


# Open info file
fpath0 = os.path.join(path_to_files, fname0)
f_info = scipy.io.loadmat(fpath0, struct_as_record=False, squeeze_me=True)
info = f_info['info']
# Open .mat file containing Calcium Imaging data
fpath1 = os.path.join(path_to_files, fname1)
file = h5py.File(fpath1, 'r')
options = file['options']
landmarkThreshold = file['landmarkThreshold']
templates = file['templates']

Y = file['Y'] 
R = file['R']
# Note: Changing axis order copies Y and R into memory
Y = np.moveaxis(Y, 1, 2) 
R = np.moveaxis(R, 1, 2)
data_shape = Y.shape

# Convert back to float32
Y = np.array(Y, dtype=np.float32)
R = np.array(R, dtype=np.float32)

In [ ]:
# Setting up compression scenarios:

# The Cartesian product of the following parameters
# will be passed for compression

# blosc compressors: 'zstd', 'blosclz','lz4','lz4hc','zlib'
# blosc compression option range: integers 1-9 
compression_opts_list = {  'lz4' : [1, 2],          # lz4 range 1-9
                           'zstd' : [1, 2],         # zstd range 1-9
                           'lzf' : [ None ],        # lzf does not take compression options
                           'gzip': [ 1, 3 ] }       # gzip compression option range: integers 1-9
shuffle_list = [1] # subset of [ hdf5plugin.Blosc.NOSHUFFLE, hdf5plugin.Blosc.SHUFFLE, hdf5plugin.Blosc.BITSHUFFLE ]
                   # or [ 0, 1, 2]

chunks_list =  [(4, 20, 10, 10),
                (4, 29, 43, 7),
                (36, 36, 10, 6),
                (4, 36, 11, 21)]

channels = { 'Channel_Y':  Y } # subset of ( Y , R ), format { 'Channel Name': Channel_Data }

In [ ]:
# Define the function to run benchmark tests

from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile, NWBHDF5IO, ProcessingModule
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, DfOverF, MotionCorrection
from pynwb.device import Device
from pynwb.base import TimeSeries
from hdmf.backends.hdf5 import H5DataIO
from IPython.display import clear_output, Markdown, update_display
import time
import warnings
warnings.simplefilter('ignore')
from axel_lab_to_nwb import SparseIterator
from axel_lab_to_nwb import generate_param_list, show_data

def run_benchmark_tests( compression_opts_list, chunks_list, shuffle_list,
                         channels, info, **kwargs ):

    display_results = show_data(channels, compression_opts_list, **kwargs)

    # create a list of cartesian product of compression options
    compression_params_list_withprogressbar = generate_param_list(compression_opts_list, chunks_list, shuffle_list)

    for compression_params in compression_params_list_withprogressbar:

        # update progressbar
        compression_params_list_withprogressbar.set_description("Processing %s" % str(compression_params) )
        compression_params_list_withprogressbar.refresh()

        # unpacking compression parameters
        compression_opts, compression, chunk_shape, shuffle = *compression_params,
        
        run_nwb_result = run_nwb(compression_opts, compression, chunk_shape, shuffle, channels, info)
        data_frame = display_results.update(run_nwb_result,
                                            compression_opts, compression, chunk_shape, shuffle, channels)
            
    return data_frame

# Collect benchmark data for a given compression scenario
# time of writing and reading data, and file size
def run_nwb(compression_opts, compression, chunks, shuffle, channels, info):

    # unpack data
    if 'Channel_Y' in channels:
        Y = channels['Channel_Y']
    if 'Channel_R' in channels:
        R = channels['Channel_R']
    
    #Create new NWB file
    nwb = NWBFile(session_description='my CaIm recording', 
                  identifier='EXAMPLE_ID',
                  session_start_time=datetime.now(tzlocal()),
                  experimenter='Evan Schaffer',
                  lab='Axel lab',
                  institution='Columbia University',
                  experiment_description='EXPERIMENT_DESCRIPTION',
                  session_id='IDX')

    #Create and add device
    device = Device('Device')
    nwb.add_device(device)

    # Create an Imaging Plane for Yellow
    optical_channel_Y = OpticalChannel(name='OpticalChannel_Y',
                                       description='2P Optical Channel',
                                       emission_lambda=510.)
    imaging_plane_Y = nwb.create_imaging_plane(name='ImagingPlane_Y',
                                               optical_channel=optical_channel_Y,
                                               description='Imaging plane',
                                               device=device,
                                               excitation_lambda=488., 
                                               imaging_rate=info.daq.scanRate,
                                               indicator='NLS-GCaMP6s',
                                               location='whole central brain')
    # Create an Imaging Plane for Red
    optical_channel_R = OpticalChannel(name='OpticalChannel_R',
                                       description='2P Optical Channel',
                                       emission_lambda=633.)
    imaging_plane_R = nwb.create_imaging_plane(name='ImagingPlane_R',
                                               optical_channel=optical_channel_R,
                                               description='Imaging plane',
                                               device=device,
                                               excitation_lambda=488., 
                                               imaging_rate=info.daq.scanRate,
                                               indicator='redStinger',
                                               location='whole central brain')

    # output file name
    fname_nwb = 'file_compressed.nwb'
    output_path_to_files = 'Folder/AxelLab/data'
    fpath_nwb = os.path.join(output_path_to_files, fname_nwb)
    if os.path.isfile(fpath_nwb):
        os.remove(fpath_nwb)

    # compression keywords to pass to h5py
    shuffle = bool(shuffle)
    if compression in ['zstd','blosclz','lz4','lz4hc','zlib']:
        compression_kw = hdf5plugin.Blosc(cname=compression, clevel=compression_opts, shuffle=shuffle)
    else:
        compression_kw = { 'compression' : compression, 'compression_opts' : compression_opts,
                           'shuffle' : shuffle }
       
    if 'Channel_Y' in channels:
        Y_chunk_iterator = SparseIterator(data=Y,
                            chunk_shape=chunks)
        Y_dataio = H5DataIO(Y_chunk_iterator, chunks=chunks, fillvalue=np.nan,
                            allow_plugin_filters = True,
                            **compression_kw)
        raw_image_series_Y = TwoPhotonSeries(name='TwoPhotonSeries_Y',
                     imaging_plane=imaging_plane_Y,
                     rate=info.daq.scanRate,
                     dimension=Y_dataio.shape,
                     unit="unit",
                     data=Y_dataio)
        nwb.add_acquisition(raw_image_series_Y)

    if 'Channel_R' in channels:
        R_chunk_iterator = SparseIterator(data=R,
                            chunk_shape=chunks)
        R_dataio = H5DataIO(R_chunk_iterator, chunks=chunks, fillvalue=np.nan,
                            allow_plugin_filters = True,
                            **compression_kw)
        raw_image_series_R = TwoPhotonSeries(name='TwoPhotonSeries_R',
                     imaging_plane=imaging_plane_R,
                     rate=info.daq.scanRate,
                     dimension=R_dataio.shape,
                     unit="unit",
                     data=R_dataio)
        nwb.add_acquisition(raw_image_series_R)

    # start compression write clock
    time_write_start = time.clock()        
        
    # Saves to NWB file
    with NWBHDF5IO(fpath_nwb, mode='w') as io:
        io.write(nwb)

    time_write = time.clock() - time_write_start

    # clear file buffer
    if os.name == 'posix':
        try:
            with open(fpath_nwb) as fdforfadvise:
                os.posix_fadvise(fdforfadvise.fileno(), 0, 0, os.POSIX_FADV_DONTNEED)
                # normal unix file buffer 
                os.posix_fadvise(fdforfadvise.fileno(), 0, 0, os.POSIX_FADV_NORMAL)
        except:
            pass
    
    # Loads NWB file
    time_read_start = time.clock()

    with NWBHDF5IO(fpath_nwb, mode='r') as io:
        nwb = io.read()
        if 'Channel_Y' in channels:
            Y_series = nwb.acquisition['TwoPhotonSeries_Y']
            # read data into memory
            Y_series_data = Y_series.data[()]
            del Y_series_data
        if 'Channel_R' in channels:
            R_series = nwb.acquisition['TwoPhotonSeries_R']
            # read data into memory
            R_series_data = R_series.data[()]
            del R_series_data

    time_read = time.clock() - time_read_start

    nwbfile_size = os.stat(fpath_nwb).st_size/1024/1024# in MB        

    # find sparse written chunks
    sparse_written_chunks = []
    if 'Channel_Y' in channels and chunks != None:
        sparse_written_chunks.append(Y_chunk_iterator.chunks_write)
    if 'Channel_R' in channels and chunks != None:
        sparse_written_chunks.append(R_chunk_iterator.chunks_write)

    return [time_write, time_read, nwbfile_size, sparse_written_chunks]


In [ ]:
# run benchmark without compression
num_uncompr_runs = 3 # 1st run is warm up, 2 for finding the average time without compression
uncompressed_results = np.zeros(num_uncompr_runs)
data_frame = run_benchmark_tests( {None: [None]}, [None]*num_uncompr_runs, [None],
                                  channels, info, show_results = False )
uncompressed_results = data_frame[["Write Time (sec)","Read Time (sec)", "File Size (MB)"]].loc[1:].mean()

# run benchmarks
df = run_benchmark_tests( compression_opts_list, chunks_list, shuffle_list, channels, info,
                         uncompressed_results = uncompressed_results, sync_volume_plot = True )
